https://habr.com/ru/companies/hh/articles/303168/

https://github.com/hhru/api/blob/master/docs/vacancies.md

https://api.hh.ru/openapi/redoc#tag/Poisk-vakansij/operation/get-vacancies

In [ ]:
!pip install pytelegrambotapi

In [258]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import date, datetime
from dotenv import load_dotenv
import telebot
import os
import time
from openai import OpenAI

In [256]:
load_dotenv()
LLM_API_KEY = os.getenv('LLM_API_KEY')
TG_BOT_TOKEN = os.getenv('TG_BOT_TOKEN')

In [316]:
def llm_prompting(token, model, temperature, system, prompt):
    client = OpenAI(base_url = "https://api.vsegpt.ru/v1", api_key = token)

    try:
        start_time = time.perf_counter()
        completion = client.chat.completions.create(
            extra_headers={},
            extra_body={},
            temperature=temperature,
            model = model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt}]
        )
        answer = completion.choices[0].message.content
        llm_input = completion.usage.prompt_tokens
        llm_output = completion.usage.completion_tokens
        end_time = time.perf_counter()
        elapsed_time = int((end_time - start_time)*1000)

        return answer, llm_input, llm_output, elapsed_time
    except Exception as e:
        return f"Ошибка: {e}", 0, 0, 0 


In [ ]:
# Загружаем существующие данные
try:
    with open('vacancies.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
except FileNotFoundError:
    data = []

# Собираем существующие id
existing_ids = {item['id'] for item in data}

In [139]:
# Пустой json для объединения всех страниц
vacancies_list = {'items': []}

url = 'https://api.hh.ru/vacancies?'
params_dict = {
    'text': 'промт',
    'per_page': '100',
    'schedule': 'fullDay',
    'work_format': 'REMOTE'
}

query_params = '&'.join(f"{key}={value}" for key, value in params_dict.items())
query = url + query_params

headers = {'User-Agent': 'api-test-agent'}

# Отключаем проверку SSL-сертификата (аналог флага -k в curl)
response = requests.get(query, headers=headers, verify=False)

vacancies_list = response.json()


pages = response.json()['pages']

print(f"Всего: {pages} страниц")
print("Страница 0 обработана")

if pages > 1:
    for i in range(pages):
        if i == 0:
            None
        else:
            global vacancies_list
            query = query + f"&page={i}"
            response = requests.get(query, headers=headers, verify=False)
            vacancies_list = {'items': vacancies_list['items'] + response.json()['items']}
            query = query.replace(f"&page={i}", "")
            print(f"Страница {i} обработана")
else:
    None

c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Всего: 4 страниц
Страница 0 обработана


c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Страница 1 обработана


c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Страница 2 обработана


c:\Users\alexe\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Страница 3 обработана


In [ ]:
# Открываем браузер
driver = webdriver.Chrome()

# Добавляем новые
for item in vacancies_list['items']:
    if item['id'] not in existing_ids:
        employer = item.get('employer', {})
        accredited_it_employer = employer.get('accredited_it_employer')
        if accredited_it_employer:
            IT = 'Да'
        else:
            IT = 'Нет'

        salary = item.get('salary', {})
        if salary != None:
            salary_from = salary.get('from')
            salary_to = salary.get('to')
            salary_cur = salary.get('currency')
        else:
            salary_from = None
            salary_to = None
            salary_cur = None

        snippet = item.get('snippet', {})

        # Переходим на страницу
        vacancy_url = item['alternate_url']
        try:
            driver.get(vacancy_url)
            # Находим элемент по атрибуту data-qa
            element = driver.find_element(By.XPATH, 
                "//*[@data-qa='vacancy-description']")
            vacancy_description = element.text
        except Exception as e:
            vacancy_description = f"Ошибка: {e}"
        
        data.append({
            'id': item['id'],
            'Название': item['name'],
            'От': salary_from,
            'До': salary_to,
            'Валюта': salary_cur,
            'Описание': vacancy_description,
            # 'Требования': snippet.get('requirement'),
            # 'Ответственность': snippet.get('responsibility'),
            'Компания': employer.get('name'),
            'IT': IT,
            'Опубликовано': item['published_at'],
            'Ссылка': vacancy_url
        })

# Закрываем браузер
driver.quit()

# Сохраняем всё
with open('vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [324]:
system_message = "Ты - большая языковая модель, личный помощник. Отвечай на вопросы пользователя на русском языке и точно по запросу."
model = 'qwen/qwen3-next-80b-a3b'
input_cost = 0.022
output_cost = 0.22

In [325]:
# Загружаем данные из файла
with open('vacancies.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Добавляем новый ключ во все элементы списка
for item in data:
    description = item['Описание']
    
    with open('prompt_eng.prmpt', 'r', encoding='utf-8') as f:
        template = f.read()

    # Интерпорлируем описание вакансии в промт
    prompt = template.format(llm_input = description)

    # Считаем число слов в промте
    item['Вход_слов'] = prompt.count(' ')

    summary, in_tokens, out_tokens, elapsed_time = llm_prompting(LLM_API_KEY, model, 0.2, system_message, prompt)
    
    item['Кратко'] = summary
    item['Вход_токенов'] = in_tokens
    item['Выход_токенов'] = out_tokens
    item['Время_генерации'] = elapsed_time
    item['Потрачено'] = input_cost*in_tokens/1000 + output_cost*out_tokens/1000

# Сохраняем обратно в файл
with open('vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [335]:
bot = telebot.TeleBot(TG_BOT_TOKEN)

with open('vacancies.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

for item in data:
    published = datetime.fromisoformat(item["Опубликовано"]).strftime("%d.%m.%Y")
    sal_f = '' if type(item["От"])!=int else f'От {item["От"]} '
    sal_t = '' if type(item["До"])!=int else f'до {item["До"]} '
    sal_c = '' if type(item["Валюта"])!=str else item["Валюта"]
    
    if sal_f==sal_t==sal_c:
        salary = 'Доход не указан'
    else:
        salary = sal_f + sal_t + sal_c

    rows = [f'*Вакансия* от {published}',
            '',
            f'[{item["Название"]}]({item["Ссылка"]})',
            salary,
            '',
            item["Кратко"],
            '',
            f'_{item["Компания"]}, IT: {item["IT"]}_',
            '',
            f'Траты: {round(item['Потрачено'],2)} ₽ | {item['Время_генерации']} мс | [Поддержать](https://tips.yandex.ru/guest/payment/3454449)'
            ]

    message = '\n'.join(row for row in rows)

    bot.send_message('@llmforall', message, parse_mode='Markdown', disable_web_page_preview=True)
    time.sleep(5)

KeyboardInterrupt: 